In [1]:
import numpy as np
import pandas as pd 
import math
import operator
%matplotlib  inline

### Split data as per problem statement

In [2]:
train_set=[0, 10]
test_set =[11,20]

### Naive Bayes model

In [3]:
class NaiveBayes:
    def __init__(self,unique_classes, expSet):
        self.globVocab = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z', ' ']
        self.classes=unique_classes
        self.clasDict={}
        self.clasVocabulary={}
        self.bagOfChar={}
        self.expSet=expSet
        self.classConditonalProbData={}
        self.bagOfCharsCountsVector={}
        self.conditonalProb={}
        self.conditionalProbabilities={}
    
    
    def getClassCharacters(self):
        for clas in self.classes:
            self.clasDict[clas]=[]
            x=[]
            for i in range(self.expSet[0],self.expSet[1]):
                filepath= clas + str(i) + '.txt'
                with open(filepath) as f:
                    x.extend([char for char in f.read().replace("\n", "")])
            # has all the characters belonging to each class as a list in sorted order
            self.clasDict[clas]=sorted(x)
     
    def getFreqOfChar(self, data):
            vocab={el:0 for el in self.globVocab}
            for char in data:
                if char in vocab:
                    vocab[char]+=1
                else:
                    vocab[char]= 1
            # create a frequency vocabulary. 
            # Has number of times each character from the set of all vocabulary appears in the respetive class
            return vocab
        
    def createClasVocaAndProbCounts(self):
        for clas in self.classes:
            sortedDictSet=sorted(set(self.clasDict[clas]))
            sortedDictSet.pop(0)
            sortedDictSet.extend([' '])
            # has the set of vocabulary ie ., a list of all unique characters
            self.clasVocabulary[clas]= sortedDictSet
            ##### check for additonal element count for add 1 smoothing
            self.bagOfChar[clas] = self.getFreqOfChar(self.clasDict[clas])
    
    def findClassCondProb(self):
        for clas in self.classes:
            den = len(self.clasDict[clas]) + len(self.clasVocabulary[clas]) 
            #print(len(self.clasDict[clas]),len(self.clasVocabulary[clas]))
            theta=[]
            for key, value in self.bagOfChar[clas].items():
                theta.extend([int(value)])
            #just vector of frequencies of char occurences in each class
            self.bagOfCharsCountsVector[clas]= theta
            #class conditional probabilities from train data
            self.classConditonalProbData[clas]=(1 + np.array(theta))/den
     
    def multinomialClassCondProb(self, testData):       
        for clas in self.classes:
            s = self.classConditonalProbData[clas]
            f = np.array(testData)
            #f = np.array(self.bagOfCharsCountsVector[clas])
            self.conditonalProb[clas] = np.dot(f, np.log(s))
            self.conditionalProbabilities[clas]= math.exp(self.conditonalProb[clas])
        print('conditonalProb exp power',self.conditonalProb)
        ##very high power to be dectected as any value
        #print('conditionalProbabilities',self.conditionalProbabilities)
        
    def totalProb(self):
        probOfX = max(self.conditonalProb.items(), key=operator.itemgetter(1))[0]
        for clas in self.classes:
            por = self.conditonalProb[clas] - self.conditonalProb[probOfX]
            print(por)
            print('posterior',math.exp(por))
    
    def bayesClassify(self):
        posteriorProb={}
        for clas in self.classes:
            posteriorProb[clas]= self.conditonalProb[clas]
            #posteriorProb[clas]= (1/3) * self.conditionalProbabilities[clas]
        maxProb=max(posteriorProb.items(), key=operator.itemgetter(1))[0]
        self.totalProb()
        return(max(posteriorProb.items(), key=operator.itemgetter(1))[0])
            
    def fetchFileData(self,fileName):
        x=[]
        filepath= fileName
        with open(filepath) as f:
            x.extend([char for char in f.read().replace("\n", "")])
        return x

    def train(self):
        self.getClassCharacters()
        self.createClasVocaAndProbCounts()
        self.findClassCondProb()
    
    def test(self):
        None

### Train Bayes Net

In [4]:


class_list =['e','j','s']
train_set=[0, 10]
test_set =[10,20]

cls=NaiveBayes(class_list, train_set)
cls.train()


## e10.txt

In [5]:

testfreqDict = cls.getFreqOfChar(sorted(cls.fetchFileData('e10.txt')))
print(testfreqDict)
theta=[]
for key, value in testfreqDict.items():
    theta.extend([int(value)])
cls.multinomialClassCondProb(theta)
testPrediction=cls.bayesClassify()
print('predicted class',testPrediction)


{'a': 164, 'b': 32, 'c': 53, 'd': 57, 'e': 311, 'f': 55, 'g': 51, 'h': 140, 'i': 140, 'j': 3, 'k': 6, 'l': 85, 'm': 64, 'n': 139, 'o': 182, 'p': 53, 'q': 3, 'r': 141, 's': 186, 't': 225, 'u': 65, 'v': 31, 'w': 47, 'x': 4, 'y': 38, 'z': 2, ' ': 498}
conditonalProb exp power {'e': -7841.78638677036, 'j': -8759.325135216695, 's': -8452.383194656028}
maxden -7841.78638677036
0.0
posterior 1.0
-917.5387484463354
posterior 0.0
-610.5968078856677
posterior 6.62484417438748e-266
predicted class e


### Test performance on test set

In [ ]:

groundTruth=[]
pred=[]
for clas in class_list:
    for inde in range(test_set[0],test_set[1]):
        filepath = clas + str(inde) + '.txt'
        groundTruth.extend(clas)
        testfreqDict = cls.getFreqOfChar(sorted(cls.fetchFileData(filepath)))
        theta=[]
        for key, value in testfreqDict.items():
            theta.extend([int(value)])
        cls.multinomialClassCondProb(theta)
        pred.extend(cls.bayesClassify())
print('ground truth classification for 10  test documents',groundTruth)
print('Predicted classification for 10  test documents',pred)

### Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(groundTruth,pred)


### Test with shuffle

In [ ]:
import random
xy=[]
with open('e10.txt') as f:
    xy.extend([char for char in f.read().replace("\n", "")])
#print(xy)
random.shuffle(xy)
print(xy)

testfreqDict = cls.getFreqOfChar(sorted(xy))
#print(testfreqDict)
theta=[]
for key, value in testfreqDict.items():
    theta.extend([int(value)])
    
print(theta)
cls.multinomialClassCondProb(theta)
testPrediction=cls.bayesClassify()
print('predicted class',testPrediction)

### Generate modified files for Weka

In [ ]:

for clas in ['e','j','s']:            
    for j in range(20):
        filepath= clas + str(j) + '.txt'
        with open(filepath) as f:
            textList = list(f.read().replace("\n", ""))
            for i in range(0, len(textList)):
                if textList[i] == " ":
                    textList[i] = "space"
            text = ' '.join(textList)
        writeToFile= clas + str(j) + 'updated.txt'
        with open(writeToFile, "w") as text_file:
            text_file.write(text)
        
